In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [ ]:
data_2018= pd.read_csv("./league-2018-2024/league_2018.csv")
data_2019= pd.read_csv("./league-2018-2024/league_2019.csv")
data_2020= pd.read_csv("./league-2018-2024/league_2020.csv")
data_2021= pd.read_csv("./league-2018-2024/league_2021.csv")
data_2022= pd.read_csv("./league-2018-2024/league_2022.csv")
data_2023= pd.read_csv("./league-2018-2024/league_2023.csv")
data_2024= pd.read_csv("./league-2018-2024/league_2024.csv")
data = pd.concat([data_2018, data_2019, data_2020, data_2021, data_2022, data_2023, data_2024], ignore_index=True)
data.shape

/tmp/ipykernel_3778/1377208622.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2022= pd.read_csv("./league-2018-2024/league_2022.csv")
/tmp/ipykernel_3778/1377208622.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2023= pd.read_csv("./league-2018-2024/league_2023.csv")


(737376, 123)

In [ ]:
for column in data.columns:
    print(column)

gameid
datacompleteness
url
league
year
split
playoffs
date
game
patch
participantid
side
position
playername
playerid
teamname
teamid
champion
ban1
ban2
ban3
ban4
ban5
gamelength
result
kills
deaths
assists
teamkills
teamdeaths
doublekills
triplekills
quadrakills
pentakills
firstblood
firstbloodkill
firstbloodassist
firstbloodvictim
team kpm
ckpm
firstdragon
dragons
opp_dragons
elementaldrakes
opp_elementaldrakes
infernals
mountains
clouds
oceans
chemtechs
hextechs
dragons (type unknown)
elders
opp_elders
firstherald
heralds
opp_heralds
firstbaron
barons
opp_barons
firsttower
towers
opp_towers
firstmidtower
firsttothreetowers
turretplates
opp_turretplates
inhibitors
opp_inhibitors
damagetochampions
dpm
damageshare
damagetakenperminute
damagemitigatedperminute
wardsplaced
wpm
wardskilled
wcpm
controlwardsbought
visionscore
vspm
totalgold
earnedgold
earned gpm
earnedgoldshare
goldspent
gspd
total cs
minionkills
monsterkills
monsterkillsownjungle
monsterkillsenemyjungle
cspm
goldat10
xpa

In [ ]:
columns_to_keep = ['league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'side', 'teamname', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'result']
df=data[columns_to_keep]
df

,league,year,split,playoffs,date,game,patch,side,teamname,ban1,ban2,ban3,ban4,ban5,result
0,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
1,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
2,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
3,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
4,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725359,NEXO,2024,Split 1,0,2023-11-20 19:59:21,2.0,13.22,Red,Erfolg Esports,Ahri,K'Sante,Maokai,Varus,Ashe,0
725360,NEXO,2024,Split 1,0,2023-11-20 19:59:21,2.0,13.22,Red,Erfolg Esports,Ahri,K'Sante,Maokai,Varus,Ashe,0
725361,NEXO,2024,Split 1,0,2023-11-20 19:59:21,2.0,13.22,Red,Erfolg Esports,Ahri,K'Sante,Maokai,Varus,Ashe,0
725362,NEXO,2024,Split 1,0,2023-11-20 19:59:21,2.0,13.22,Blue,Kawaii Kiwis,Seraphine,Jayce,Jarvan IV,Vi,Syndra,1


In [ ]:
player_names_list = []
champion_names_list = []

# Initialize variables to keep track of the current row
current_row = 0

while current_row < len(data):
    # Extract player names and champion names from rows 1 to 10
    player_names = data.iloc[current_row:current_row + 10]['playername'].tolist()
    champion_names = data.iloc[current_row:current_row + 10]['champion'].tolist()
    
    # Add the player names and champion names to the lists
    player_names_list.extend(player_names)
    champion_names_list.extend(champion_names)
    
    # Move to the next team and skip 2 rows
    current_row += 12

# Create a dictionary to store the data
player_data = {}
for i in range(1, 11):
    player_data[f'playername_{i}'] = player_names_list[i-1::10]
    player_data[f'champion_{i}'] = champion_names_list[i-1::10]

new_df = pd.DataFrame(player_data)
new_df

,playername_1,champion_1,playername_2,champion_2,playername_3,champion_3,playername_4,champion_4,playername_5,champion_5,playername_6,champion_6,playername_7,champion_7,playername_8,champion_8,playername_9,champion_9,playername_10,champion_10
0,Zoom,Shen,Clid,Kha'Zix,Yagao,Azir,LokeN,Jhin,LvMao,Alistar,TheShy,Camille,Ning,Rek'Sai,Rookie,Zoe,JackeyLove,Xayah,Baolan,Rakan
1,TheShy,Trundle,Ning,Jarvan IV,Rookie,Azir,JackeyLove,Caitlyn,Baolan,Tahm Kench,Zoom,Shen,Clid,Sejuani,Yagao,Zoe,LokeN,Jhin,LvMao,Taric
2,AmazingJ,Gangplank,Wisdomz,Gragas,Athena,Taliyah,Jinjiao,Miss Fortune,Road,Tahm Kench,Flandre,Camille,SofM,Kha'Zix,Guoguo,Orianna,kRYST4L,Jhin,Hudie,Shen
3,Flandre,Vladimir,SofM,Jarvan IV,Guoguo,Azir,kRYST4L,Kalista,Hudie,Braum,AmazingJ,Camille,Wisdomz,Sejuani,Athena,Xerath,Jinjiao,Jhin,Road,Shen
4,Mouse,Gnar,Flawless,Sejuani,Doinb,Kassadin,Smlz,Miss Fortune,Killua,Tahm Kench,Ray,Maokai,Haro,Jarvan IV,Scout,Zoe,iBoy,Kalista,Meiko,Braum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60442,Parlita,Fiora,Tikyl,Vi,Piku,Syndra,Marcv1,Ziggs,Kandar,Rakan,Jervo,Malphite,Nasser,Graves,Zherathor,Neeko,Aaron,Aphelios,Aizhon,Rell
60443,Jervo,Poppy,Nasser,Rell,Zherathor,Orianna,Aaron,Varus,Aizhon,Renata Glasc,Parlita,Gwen,Tikyl,Lee Sin,Piku,Syndra,Marcv1,Aphelios,Kandar,Nautilus
60444,Parlita,Gnar,Tikyl,Poppy,Piku,Syndra,Marcv1,Aphelios,Kandar,Nautilus,Jervo,Sion,Nasser,Graves,Zherathor,Azir,Aaron,Kalista,Aizhon,Rell
60445,Dani,Aatrox,Emvipi,Rell,Nestor,Orianna,Kurilius,Lucian,Whyx,Nautilus,DoubleB,Camille,Ivama,Wukong,SERCH,Syndra,F3IV,Ziggs,EscoX,Rakan


In [ ]:
rows_to_delete = []

# Initialize variables to keep track of the current row
current_row = 0

while current_row < len(df):
    # Identify the indices of rows to be deleted (10 consecutive rows)
    rows_to_delete.extend(list(range(current_row, current_row + 10)))
    
    # Move to the next set of rows to keep (2 consecutive rows)
    current_row += 12

# Drop the rows to be deleted from the original DataFrame
teams_df = df.drop(rows_to_delete)

# Reset the index of the modified DataFrame
teams_df.reset_index(drop=True, inplace=True)
teams_df

,league,year,split,playoffs,date,game,patch,side,teamname,ban1,ban2,ban3,ban4,ban5,result
0,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,JD Gaming,NaN,NaN,NaN,NaN,NaN,0
1,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Red,Invictus Gaming,NaN,NaN,NaN,NaN,NaN,1
2,DCup,2017,NaN,0,2018-01-04 04:31:10,2.0,7.24,Blue,Invictus Gaming,Ornn,Ezreal,Kalista,Gangplank,Maokai,1
3,DCup,2017,NaN,0,2018-01-04 04:31:10,2.0,7.24,Red,JD Gaming,Miss Fortune,Ryze,Camille,Gnar,Jayce,0
4,DCup,2017,NaN,0,2018-01-04 05:56:28,1.0,7.24,Blue,Bilibili Gaming,Ryze,Ornn,Jarvan IV,LeBlanc,Draven,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120889,NEXO,2024,Split 1,0,2023-11-20 17:50:57,3.0,13.22,Red,Dango SB,K'Sante,Xayah,Orianna,Jax,Gwen,0
120890,NEXO,2024,Split 1,0,2023-11-20 19:05:47,1.0,13.22,Blue,Kawaii Kiwis,Seraphine,Jayce,Jarvan IV,Viego,Nocturne,1
120891,NEXO,2024,Split 1,0,2023-11-20 19:05:47,1.0,13.22,Red,Erfolg Esports,Ahri,K'Sante,Maokai,Vi,Poppy,0
120892,NEXO,2024,Split 1,0,2023-11-20 19:59:21,2.0,13.22,Blue,Kawaii Kiwis,Seraphine,Jayce,Jarvan IV,Vi,Syndra,1


In [ ]:
output_data = []

# Initialize variables to keep track of the current row
current_row = 0

while current_row < len(teams_df):
    # Get information from the first row in the pair
    row1 = teams_df.iloc[current_row]

    # Check if 'result' in the first row is 1

    # Get information from the second row in the pair
    row2 = teams_df.iloc[current_row + 1]
    if row1['result'] == 1:
        winning_team = row1['teamname']
    else:
        winning_team = row2['teamname']
    # Combine information into a dictionary for the output
    output_row = {
        'league': row1['league'],
        'year': row1['year'],
        'split': row1['split'],
        'playoffs': row1['playoffs'],
        'date': row1['date'],
        'game': row1['game'],
        'patch': row1['patch'],
        't1_side': row1['side'],
        't2_side': row2['side'],
        'teamname_1': row1['teamname'],
        'teamname_2': row2['teamname'],
        'ban1': row1['ban1'],
        'ban2': row1['ban2'],
        'ban3': row1['ban3'],
        'ban4': row1['ban4'],
        'ban5': row1['ban5'],
        't2_ban1': row2['ban1'],
        't2_ban2': row2['ban2'],
        't2_ban3': row2['ban3'],
        't2_ban4': row2['ban4'],
        't2_ban5': row2['ban5'],
        'winning_team': winning_team
    }

    # Append the output row to the list
    output_data.append(output_row)

    # Move to the next pair of rows
    current_row += 2

# Create the output DataFrame
output_df = pd.DataFrame(output_data)
output_df

,league,year,split,playoffs,date,game,patch,t1_side,t2_side,teamname_1,...,ban2,ban3,ban4,ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,winning_team
0,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,Blue,Red,JD Gaming,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invictus Gaming
1,DCup,2017,NaN,0,2018-01-04 04:31:10,2.0,7.24,Blue,Red,Invictus Gaming,...,Ezreal,Kalista,Gangplank,Maokai,Miss Fortune,Ryze,Camille,Gnar,Jayce,Invictus Gaming
2,DCup,2017,NaN,0,2018-01-04 05:56:28,1.0,7.24,Blue,Red,Bilibili Gaming,...,Ornn,Jarvan IV,LeBlanc,Draven,Azir,Ezreal,Sejuani,Rengar,Shyvana,Snake Esports
3,DCup,2017,NaN,0,2018-01-04 06:53:17,2.0,7.24,Blue,Red,Snake Esports,...,Miss Fortune,Gangplank,Malzahar,Kassadin,Ornn,Ryze,Tahm Kench,Taric,Janna,Snake Esports
4,DCup,2017,NaN,0,2018-01-04 08:26:00,1.0,7.24,Blue,Red,Rogue Warriors,...,Vladimir,Ezreal,Thresh,Shen,Ryze,Camille,Ornn,Malzahar,Jayce,EDward Gaming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60442,NEXO,2024,Split 1,0,2023-11-20 16:07:53,1.0,13.22,Blue,Red,Oxygen Kumiho,...,Rumble,Jarvan IV,Nocturne,Senna,K'Sante,Xayah,Orianna,Nautilus,Alistar,Dango SB
60443,NEXO,2024,Split 1,0,2023-11-20 16:59:21,2.0,13.22,Blue,Red,Dango SB,...,K'Sante,Xayah,Vi,Nocturne,LeBlanc,Rumble,Jarvan IV,Graves,Viego,Oxygen Kumiho
60444,NEXO,2024,Split 1,0,2023-11-20 17:50:57,3.0,13.22,Blue,Red,Oxygen Kumiho,...,LeBlanc,Rumble,Varus,Malphite,K'Sante,Xayah,Orianna,Jax,Gwen,Oxygen Kumiho
60445,NEXO,2024,Split 1,0,2023-11-20 19:05:47,1.0,13.22,Blue,Red,Kawaii Kiwis,...,Jayce,Jarvan IV,Viego,Nocturne,Ahri,K'Sante,Maokai,Vi,Poppy,Kawaii Kiwis


In [ ]:
champion_mapping = {
    'Aatrox': 1,
    'Ahri': 2,
    'Akali': 3,
    'Akshan': 156,
    'Alistar': 4,
    'Amumu': 5,
    'Anivia': 6,
    'Annie': 7,
    'Aphelios': 8,
    'Ashe': 9,
    'Aurelion Sol': 10,
    'Azir': 11,
    'Bard': 12,
    'Bel\'Veth': 157,
    'Blitzcrank': 13,
    'Brand': 14,
    'Braum': 15,
    'Briar': 158,
    'Caitlyn': 16,
    'Camille': 17,
    'Cassiopeia': 18,
    'Cho\'Gath': 19,
    'Corki': 20,
    'Darius': 21,
    'Diana': 22,
    'Dr. Mundo': 23,
    'Draven': 24,
    'Ekko': 25,
    'Elise': 26,
    'Evelynn': 27,
    'Ezreal': 28,
    'Fiddlesticks': 29,
    'Fiora': 30,
    'Fizz': 31,
    'Galio': 32,
    'Gangplank': 33,
    'Garen': 34,
    'Gnar': 35,
    'Gragas': 36,
    'Graves': 37,
    'Gwen': 154,
    'Hecarim': 38,
    'Heimerdinger': 39,
    'Illaoi': 40,
    'Irelia': 41,
    'Ivern': 42,
    'Janna': 43,
    'Jarvan IV': 44,
    'Jax': 45,
    'Jayce': 46,
    'Jhin': 47,
    'Jinx': 48,
    'K\'Sante':158,
    'Kai\'Sa': 49,
    'Kalista': 50,
    'Karma': 51,
    'Karthus': 52,
    'Kassadin': 53,
    'Katarina': 54,
    'Kayle': 55,
    'Kayn': 56,
    'Kennen': 57,
    'Kha\'Zix': 58,
    'Kindred': 59,
    'Kled': 60,
    'Kog\'Maw': 61,
    'LeBlanc': 62,
    'Lee Sin': 63,
    'Leona': 64,
    'Lillia': 65,
    'Lissandra': 66,
    'Lucian': 67,
    'Lulu': 68,
    'Lux': 69,
    'Malphite': 70,
    'Malzahar': 71,
    'Maokai': 72,
    'Master Yi': 73,
    'Milio':159,
    'Miss Fortune': 74,
    'Mordekaiser': 75,
    'Morgana': 76,
    'Naafiri': 160,
    'Nami': 77,
    'Nasus': 78,
    'Nautilus': 79,
    'Neeko': 80,
    'Nidalee': 81,
    'Nilah': 161,
    'Nocturne': 82,
    'Nunu & Willump': 83,
    'Olaf': 84,
    'Orianna': 85,
    'Ornn': 86,
    'Pantheon': 87,
    'Poppy': 88,
    'Pyke': 89,
    'Qiyana': 90,
    'Quinn': 91,
    'Rakan': 92,
    'Rammus': 93,
    'Rek\'Sai': 94,
    'Rell':162,
    'Renata Glasc':163,
    'Renekton': 95,
    'Rengar': 96,
    'Riven': 97,
    'Rumble': 98,
    'Ryze': 99,
    'Samira': 100,
    'Sejuani': 101,
    'Senna': 102,
    'Seraphine': 103,
    'Sett': 104,
    'Shaco': 105,
    'Shen': 106,
    'Shyvana': 107,
    'Singed': 108,
    'Sion': 109,
    'Sivir': 110,
    'Skarner': 111,
    'Sona': 112,
    'Soraka': 113,
    'Swain': 114,
    'Sylas': 115,
    'Syndra': 116,
    'Tahm Kench': 117,
    'Taliyah': 118,
    'Talon': 119,
    'Taric': 120,
    'Teemo': 121,
    'Thresh': 122,
    'Tristana': 123,
    'Trundle': 124,
    'Tryndamere': 125,
    'Twisted Fate': 126,
    'Twitch': 127,
    'Udyr': 128,
    'Urgot': 129,
    'Varus': 130,
    'Vayne': 131,
    'Veigar': 132,
    'Vel\'Koz': 133,
    'Vex': 155,
    'Vi': 134,
    'Viego': 135,
    'Viktor': 136,
    'Vladimir': 137,
    'Volibear': 138,
    'Warwick': 139,
    'Wukong': 140,
    'Xayah': 141,
    'Xerath': 142,
    'Xin Zhao': 143,
    'Yasuo': 144,
    'Yone': 145,
    'Yorick': 146,
    'Yuumi': 147,
    'Zac': 148,
    'Zed': 149,
    'Zeri': 150,
    'Ziggs': 164,
    'Zilean': 151,
    'Zoe': 152,
    'Zyra': 153,
    'Hwei':154,
    'Smolder': 155
}

In [ ]:
merged_df = pd.concat([output_df, new_df], axis=1)
merged_df['champion_1'] = merged_df['champion_1'].map(champion_mapping)
merged_df['champion_2'] = merged_df['champion_2'].map(champion_mapping)
merged_df['champion_3'] = merged_df['champion_3'].map(champion_mapping)
merged_df['champion_4'] = merged_df['champion_4'].map(champion_mapping)
merged_df['champion_5'] = merged_df['champion_5'].map(champion_mapping)
merged_df['champion_6'] = merged_df['champion_6'].map(champion_mapping)
merged_df['champion_7'] = merged_df['champion_7'].map(champion_mapping)
merged_df['champion_8'] = merged_df['champion_8'].map(champion_mapping)
merged_df['champion_9'] = merged_df['champion_9'].map(champion_mapping)
merged_df['champion_10'] = merged_df['champion_10'].map(champion_mapping)

columns_to_check = ['champion_1', 'champion_2', 'champion_3', 'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9', 'champion_10']

# Iterate through the specified columns and check for null values in each column
merged_df = merged_df.dropna(subset=columns_to_check)
print(merged_df.columns)

Index(['league', 'year', 'split', 'playoffs', 'date', 'game', 'patch',
       't1_side', 't2_side', 'teamname_1', 'teamname_2', 'ban1', 'ban2',
       'ban3', 'ban4', 'ban5', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4',
       't2_ban5', 'winning_team', 'playername_1', 'champion_1', 'playername_2',
       'champion_2', 'playername_3', 'champion_3', 'playername_4',
       'champion_4', 'playername_5', 'champion_5', 'playername_6',
       'champion_6', 'playername_7', 'champion_7', 'playername_8',
       'champion_8', 'playername_9', 'champion_9', 'playername_10',
       'champion_10'],
      dtype='object')


In [ ]:
player_name_mapping = {}
number = 1
for col in merged_df.columns:
    if col.startswith('playername_'):
        player_names = merged_df[col].unique()
        for name in player_names:
            if name not in player_name_mapping:
                player_name_mapping[name] = number
                number += 1

# Replace player names with numbers in columns playername_1 to playername_10
for col in merged_df.columns:
    if col.startswith('playername_'):
        merged_df[col] = merged_df[col].map(player_name_mapping)


In [ ]:
team_name_mapping = {}
number = 1

for col in ['teamname_1', 'teamname_2']:
    team_names = merged_df[col].unique()
    for name in team_names:
        if name not in team_name_mapping:
            team_name_mapping[name] = number
            number += 1

# Replace team names with numbers in columns teamname_1 and teamname_2
for col in ['teamname_1', 'teamname_2']:
    merged_df[col] = merged_df[col].map(team_name_mapping)
merged_df['winning_team'] = merged_df['winning_team'].map(team_name_mapping)
merged_df['winning_team'] = merged_df.apply(lambda row: 0 if row['winning_team'] == row['teamname_1'] else 1, axis=1)
value_counts = merged_df['winning_team'].value_counts()

# Print the counts
print(value_counts)

winning_team
0    32066
1    28381
Name: count, dtype: int64


In [ ]:
side_mapping = {'Blue': 0, 'Red': 1}
merged_df['t1_side'] = merged_df['t1_side'].map(side_mapping)
merged_df['t2_side'] = merged_df['t2_side'].map(side_mapping)
merged_df

,league,year,split,playoffs,date,game,patch,t1_side,t2_side,teamname_1,...,playername_6,champion_6,playername_7,champion_7,playername_8,champion_8,playername_9,champion_9,playername_10,champion_10
0,DCup,2017,NaN,0,2018-01-04 03:23:00,1.0,7.24,0,1,1,...,2,17,1717,94,3397,152,4790,141,6176,92
1,DCup,2017,NaN,0,2018-01-04 04:31:10,2.0,7.24,0,1,2,...,1,106,1716,101,3396,152,4789,47,6175,120
2,DCup,2017,NaN,0,2018-01-04 05:56:28,1.0,7.24,0,1,3,...,4,17,1719,58,3399,85,4792,47,5049,106
3,DCup,2017,NaN,0,2018-01-04 06:53:17,2.0,7.24,0,1,4,...,3,17,1718,101,3398,142,4791,47,4937,106
4,DCup,2017,NaN,0,2018-01-04 08:26:00,1.0,7.24,0,1,5,...,6,72,1721,44,3401,152,4794,50,6178,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60442,NEXO,2024,Split 1,0,2023-11-20 16:07:53,1.0,13.22,0,1,1448,...,855,70,2298,37,4013,80,5301,8,6660,162
60443,NEXO,2024,Split 1,0,2023-11-20 16:59:21,2.0,13.22,0,1,1433,...,1536,154,3230,63,2812,116,5973,8,5629,79
60444,NEXO,2024,Split 1,0,2023-11-20 17:50:57,3.0,13.22,0,1,1448,...,855,109,2298,37,4013,11,5301,50,6660,162
60445,NEXO,2024,Split 1,0,2023-11-20 19:05:47,1.0,13.22,0,1,521,...,1710,17,3305,140,1540,116,6168,164,7003,92


In [ ]:
league_mapping = {}
number = 1

league_names = merged_df['league'].unique()
for name in league_names:
    if name not in league_mapping:
        league_mapping[name] = number
        number += 1

# Replace league names with numbers in the 'league' column
merged_df['league'] = merged_df['league'].map(league_mapping)

In [ ]:
columns_to_replace = ['ban1', 'ban2', 'ban3', 'ban4', 'ban5', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5']

# Iterate over columns and replace values
for col in columns_to_replace:
    merged_df[col] = merged_df[col].replace(champion_mapping)

# Drop rows with NaN values in the specified columns and assign the result back to merged_df
merged_df = merged_df.dropna(subset=columns_to_replace)

In [ ]:
merged_df= merged_df.drop(['date'], axis=1)

In [ ]:
split_mapping ={}
num =1
unique_values = merged_df['split'].dropna().unique()
for col in unique_values:
    if col not in split_mapping:
        split_mapping[col] = num
        num +=1
merged_df['split'] = merged_df['split'].map(split_mapping).fillna(0)
merged_df

,league,year,split,playoffs,game,patch,t1_side,t2_side,teamname_1,teamname_2,...,playername_6,champion_6,playername_7,champion_7,playername_8,champion_8,playername_9,champion_9,playername_10,champion_10
1,1,2017,0.0,0,2.0,7.24,0,1,2,1,...,1,106,1716,101,3396,152,4789,47,6175,120
2,1,2017,0.0,0,1.0,7.24,0,1,3,4,...,4,17,1719,58,3399,85,4792,47,5049,106
3,1,2017,0.0,0,2.0,7.24,0,1,4,3,...,3,17,1718,101,3398,142,4791,47,4937,106
4,1,2017,0.0,0,1.0,7.24,0,1,5,6,...,6,72,1721,44,3401,152,4794,50,6178,15
5,1,2017,0.0,0,2.0,7.24,0,1,6,5,...,5,72,1720,58,3400,71,4793,47,6177,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60442,55,2024,2.0,0,1.0,13.22,0,1,1448,1433,...,855,70,2298,37,4013,80,5301,8,6660,162
60443,55,2024,2.0,0,2.0,13.22,0,1,1433,1448,...,1536,154,3230,63,2812,116,5973,8,5629,79
60444,55,2024,2.0,0,3.0,13.22,0,1,1448,1433,...,855,109,2298,37,4013,11,5301,50,6660,162
60445,55,2024,2.0,0,1.0,13.22,0,1,521,1447,...,1710,17,3305,140,1540,116,6168,164,7003,92


In [ ]:
merged_df = merged_df.dropna(subset=['game'])
merged_df = merged_df.dropna(subset=['patch'])

In [ ]:
y = merged_df.winning_team
X= merged_df.drop(['winning_team'], axis=1)

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,
                                                                random_state=0)

In [ ]:
xgb_final = xgb.XGBClassifier(max_depth=6,
                          learning_rate=0.05,
                          n_estimators=200,  
                          min_child_weight=2,
                          scale_pos_weight=0.5,
                          subsample=0.9 ,
                          colsample_bytree=0.5,
                          colsample_bylevel=0.8 ,
                          reg_alpha=0.05 ,
                          reg_lambda=0.1 ,
                          max_delta_step=2 ,
                          gamma=0.1,
                          random_state=0)
xgb_final.fit(X_train_full, y_train)
y_pred = xgb_final.predict(X_valid_full)
precision_2 = precision_score(y_valid, y_pred)

In [ ]:
print(precision_2)

0.7539766702014846


In [ ]:
SEED=43
np.random.seed(SEED)

In [ ]:
kf = StratifiedKFold(
    n_splits=10, 
    shuffle=True, 
    random_state=SEED)

In [ ]:
# got them from @sarun pm + some changes
xboost_params = {
    'n_estimators': 1499,
    'max_depth': 3,
    'learning_rate': 0.1423083245472853,
    'reg_lambda': 0.03455727849711498,
    'reg_alpha': 0.8679615702388653,
    'gamma': 0.26472758167540117,
    'subsample': 0.6100000000000001, 
    'min_child_weight': 41.901424395832386,
    'colsample_bytree': 0.6523332414965578,
    'colsample_bylevel': 0.9212808267336932,
    'colsample_bynode': 0.9800398595585802,
    'objective': 'binary:logistic',
    'random_state': SEED,
    'eval_metric': 'error', 
    'tree_method': 'hist',
    'lambda': 0.0004422864343763829,
    'alpha': 2.153918018817405e-06,
    'grow_policy': 'lossguide',
    'eta': 0.3,
}

In [ ]:
def kfold_fit(model_constructor, model_params, cv_folds, feature_data, target_data, model_name='XGBoost'):
    
    # Create session dict
    session = {
        'scores': [],
        'predictions': [],
        'models': [],
        'Name': []
    }

    # Train models on folds
    print(f'Training {model_name} Models')
    for fold, (train_idx, val_idx) in enumerate(cv_folds.split(feature_data, target_data)):
        print('----------------------------------')
        print(f"Training on Fold {fold + 1} ... \n")

        # Get Fold Training and Validation Split Data
        X_train, X_val = feature_data.iloc[train_idx], feature_data.iloc[val_idx]
        y_train, y_val = target_data.iloc[train_idx], target_data.iloc[val_idx]

        # Generate model
        model = model_constructor(**model_params)

        # Fit the model
        start = time.perf_counter()
        model.fit(X_train, y_train.ravel())
        end = time.perf_counter()

        # Compute total training time
        total_training_time = end - start

        # Make predictions
        val_predictions = model.predict(X_val).ravel()

        # Calculate accuracy
        val_score = accuracy_score(y_val, val_predictions)
        session['scores'].append(val_score)

        # Print Training Time & Accuracy
        print(f'Time: {total_training_time:.2f}s\t| Accuracy: {val_score:.6f}')

        # Record Model
        session['models'].append(model)

    # Print OOF Accuracy score
    print('==================================')
    print(f"\nOOF Accuracy: {np.mean(session['scores']):.6f}\n")
    print('==================================')
    
    return session

In [ ]:
xgboost_training = kfold_fit(XGBClassifier, 
                        xboost_params,
                        kf, 
                        X, 
                        y,
                        model_name='Optimized XGBoost')
xgboost_preds = np.mean(np.array(xgboost_training['predictions']), axis=0).ravel()

Training Optimized XGBoost Models
----------------------------------
Training on Fold 1 ... 

Time: 3.11s	| Accuracy: 0.609950
----------------------------------
Training on Fold 2 ... 

Time: 2.96s	| Accuracy: 0.622560
----------------------------------
Training on Fold 3 ... 

Time: 3.28s	| Accuracy: 0.605286
----------------------------------
Training on Fold 4 ... 

Time: 2.87s	| Accuracy: 0.614787
----------------------------------
Training on Fold 5 ... 

Time: 2.70s	| Accuracy: 0.609259
----------------------------------
Training on Fold 6 ... 

Time: 3.14s	| Accuracy: 0.621178
----------------------------------
Training on Fold 7 ... 

Time: 2.83s	| Accuracy: 0.615930
----------------------------------
Training on Fold 8 ... 

Time: 2.71s	| Accuracy: 0.601417
----------------------------------
Training on Fold 9 ... 

Time: 2.67s	| Accuracy: 0.619385
----------------------------------
Training on Fold 10 ... 

Time: 3.53s	| Accuracy: 0.611956

OOF Accuracy: 0.613171



/home/smilox/miniconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/smilox/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
